Reading ChEMBL data...
2496335 processed compounds
Selecting only small molecules...
1915466 selected small molecules
Subsampling 500000 small molecules...
Calculating Morgan Fingerprints


100%|██████████| 500000/500000 [02:04<00:00, 4015.28it/s]


In [3]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)